In [3]:
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np
import itertools
import struct
import pandas as pd
import time
import gc
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
from tqdm import tqdm, trange
from torch.utils.data import DataLoader, TensorDataset
import pynvml
import time

In [4]:
maxI,minI,maxV,minV,maxT,minT=[9.381874,-20.00478,3.600254,1.989369,57.09709,18.68196]

In [5]:
dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

In [6]:
def Sequence_Split_SOC(df,X,Y):
    for i in range(0,500):
        for j in range(0,len(df["SOC"][i])-100):
            I=df["I[A]"][i][j:j+100]
            V=df["U[V]"][i][j:j+100]
            T=df["T1[°C]"][i][j:j+100]
            if(len(I)==100):
                Y.append(df["SOC"][i][j+100])
                X.append([I,V,T])


def Sequence_Split_SOH(sequence,X,y,step_in,step_out):

   for i in range(len(sequence)):
       end_ix = i + step_in
       out_end_ix = end_ix + step_out

       if out_end_ix > len(sequence):
              break
 # gather input and output parts of the pattern
       seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
       X.append(seq_x.drop(["SOC","SOH","Qd","QC","Time[h]","Cycle_index"],axis=1).to_numpy())
       y.append(seq_y["SOH"].to_numpy())
   return X, y

def Normalization(data,stepin,maxlength=1500):
    nd=[]
    for i in range(len(data)):
        im=[]
        vm=[]
        tm=[]
        for c in range(len(data[i])):
            norm_i = (data[i][c][0] - minI) / (maxI - minI)
            norm_v = (data[i][c][1] - minV) / (maxV - minV)
            norm_t = (data[i][c][2] - minT) / (maxT - minT)
            t=np.zeros((1500,))
            t[0:min(1500,len(norm_v))]=norm_i[:min(1500,len(norm_i))]
            im.append(t)

            t=np.zeros((1500,))
            t[0:min(1500,len(norm_v))]=norm_v[:min(1500,len(norm_v))]
            vm.append(t)

            t=np.zeros((1500,))
            t[0:min(1500,len(norm_v))]=norm_t[:min(1500,len(norm_t))]
            tm.append(t)
        d=np.float32([im,vm,tm])
        d=np.reshape(np.transpose(np.reshape(d,(3,stepin*1500))),(1500*stepin,3))
        nd.append(d)
    return nd
def Encoding(data,stpsin):
    nd= []
    r=(stpsin*1500)-1
    for i in range(len(data)):
        current = torch.tensor(data[i], dtype=torch.float32, device="cuda")  
        dffi = torch.abs(torch.diff(current[:,0]))
        dffv = torch.abs(torch.diff(current[:,1]))
        dfft = torch.abs(torch.diff(current[:,2]))
        im_seq = (dffi >= 0.0002).int()
        vm_seq = (dffv >= 0.0002).int()
        tm_seq = (dfft >= 0.002).int()
        d=np.float32([im_seq.detach().cpu().numpy(),vm_seq.detach().cpu().numpy(),tm_seq.detach().cpu().numpy()])
        d=np.reshape(np.transpose(np.reshape(d,(3,r))),(r,3))
        nd.append(d)
    return nd

loss2=nn.MSELoss()
loss=nn.L1Loss()
def measure_accuracy(model, dataloader):
  t=[]
  with torch.no_grad():
    model.to(device)
    model.eval()
    for data, targets in iter(dataloader):
      data = data.to(device)
      targets = targets.to(device)

      # forward-pass
      t1=time.time()
      spk_rec= model(data)
      t2=time.time()
      t.append(t2-t1)
      loss_mae= torch.zeros((1), dtype=dtype, device=device)
      loss_mse= torch.zeros((1), dtype=dtype, device=device)
      loss_rmse= torch.zeros((1), dtype=dtype, device=device)
      loss_mae = loss(spk_rec, targets)
      loss_mse=loss2(spk_rec, targets) 
      loss_rmse=torch.sqrt(loss_mse)
    return loss_mae,loss_mse,loss_rmse,np.average(t)
            



In [7]:
class Reshape(nn.Module):
    def __init__(self, shape):
        super().__init__()
        self.shape = shape  

    def forward(self, x):
        return x.view(x.size(0), *self.shape)

class GRU_CNN(nn.Module):
    def __init__(self,stepin,stpout):
        super(GRU_CNN, self).__init__()

        self.conv_block = nn.Sequential(
            Reshape((3, stepin*1500)),  # From (batch, 37500, 3) ➜ (batch, 3, 37500)

            nn.Conv1d(3, 64, kernel_size=32, padding='same'),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),

          
            nn.AdaptiveAvgPool1d(4)
        )

        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(64 * 4, 64)  

        self.gru = nn.GRU(3, 256, batch_first=True)
        self.dense2 = nn.Linear(256, 64)

        self.concat = nn.Linear(128, stpout)  

    def forward(self, input_stream):
        x1 = self.conv_block(input_stream)
        x1 = self.flatten(x1)
        x1 = self.dense1(x1)
        _, x2 = self.gru(input_stream)
        x2 = x2.squeeze(0)
        x2 = self.dense2(x2)

        combined = torch.cat((x1, x2), dim=1)
        output = self.concat(combined)

        return output

In [16]:
def Test_Model(Steps_in,Steps_out):
    
    for stpsin,stpout in tqdm(zip(Steps_in,Steps_out)):

        data = np.load(f"Data\\Xtrain_normilized_{stpsin}_{stpout}.npz")
        X1 = data['Data']
        data = np.load(f"Data\\Ytrain_{stpsin}_{stpout}.npz")
        Y1 = data['y']
        print(f"Trainning will be on {device}")


        X = np.float32(X1)
        Y= np.float32(Y1)
        Y=np.reshape(Y,(len(Y),stpout))

        x_tensor = torch.tensor(X)
        y_tensor = torch.tensor(Y)




        dataset = TensorDataset(x_tensor, y_tensor)

        batch_size = 32  
        train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
        model = GRU_CNN(stpsin,stpout).to(device)
        #summary(model, input_size=(1500*stpsin, 3)) 
        data = np.load(f"Data\\Xtest_normilized_{stpsin}_{stpout}.npz")
        X2 = data['Data']
        data = np.load(f"Data\\Ytest_{stpsin}_{stpout}.npz")
        Y2 = data['y']
        X = np.float32(X2)
        Y= np.float32(Y2)
        Y=np.reshape(Y,(len(Y),stpout))

        x_tensor = torch.tensor(X)
        y_tensor = torch.tensor(Y)

        dataset = TensorDataset(x_tensor, y_tensor)


        batch_size = 64  
        test_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

        num_epochs = 1
        loss_hist = []
        test_loss_hist = []
        counter = 0
        loss = nn.L1Loss()
        num_steps=1
        optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999))
        # Outer training loop
        for epoch in range(num_epochs):
            iter_counter = 0
            train_batch = iter(train_loader)
            # Minibatch training loop
            for data, targets in train_batch:
                data = data.to(device)
                targets = targets.to(device)

                # forward pass
                model.train()
                spk_rec = model(data)

                # initialize the loss & sum over time
                loss_val = torch.zeros((1), dtype=dtype, device=device)
                for step in range(num_steps):
                    loss_val += loss(spk_rec, targets)

                # Gradient calculation + weight update
                optimizer.zero_grad()
                loss_val.backward()
                optimizer.step()

                # Store loss history for future plotting
                loss_hist.append(loss_val.item())

                # Test set
                with torch.no_grad():
                    model.eval()
                    test_data, test_targets = next(iter(test_loader))
                    test_data = test_data.to(device)
                    test_targets = test_targets.to(device)

                    # Test set forward pass
                    test_spk= model(test_data)

                    # Test set loss
                    test_loss = torch.zeros((1), dtype=dtype, device=device)
                    for step in range(num_steps):
                        test_loss += loss(test_spk, test_targets)
                    test_loss_hist.append(test_loss.item())
                    counter += 1
                    iter_counter +=1


        torch.save(model.state_dict(), f'Models\\modelGRU_weights_{stpsin}_{stpout}.pth')
        loss_mae,loss_mse,loss_rmse,tttime=measure_accuracy(model, test_loader)
        input = torch.randn(1, 1500*stpsin, 3).to('cuda')
        pynvml.nvmlInit()
        handle = pynvml.nvmlDeviceGetHandleByIndex(0)

        start = time.time()
        power_start = pynvml.nvmlDeviceGetPowerUsage(handle)  # in mW

        # Run your inference
        output = model(input)

        power_end = pynvml.nvmlDeviceGetPowerUsage(handle)
        end = time.time()

        avg_power = (power_start + power_end) / 2  # in milliwatts
        duration = end - start  # in seconds

        energy_mJ = avg_power * duration  # mJ = mW * s

        print("MAE",loss_mae,"MSE",loss_mse,"RMSE",loss_rmse,"RunningTime",np.mean(tttime),"Energy",energy_mJ) 

        results=[loss_mae.item(),loss_mse.item(),loss_rmse.item(),tttime,energy_mJ]
        np.save(f"Results\\resultsGRU_{stpsin}_{stpout}",results)

        del model, optimizer, train_loader, test_loader
        del X, Y, X1, X2, Y1, Y2, x_tensor, y_tensor
        del dataset, data, targets, spk_rec
        del test_spk, test_data, test_targets
        del loss_val, test_loss, loss_hist, test_loss_hist
        
        # Free CUDA memory
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            torch.cuda.synchronize()

        # Free CPU memory
        gc.collect()





In [17]:
Steps_in=[1,10,20,25,25,25,25]
Steps_out=[1,5,10,25,50,100,200]

Test_Model(Steps_in,Steps_out)

0it [00:00, ?it/s]

Trainning will be on cuda
MAE tensor(0.1696, device='cuda:0') MSE tensor(0.0370, device='cuda:0') RMSE tensor(0.1925, device='cuda:0') RunningTime 1.8454160196287137 Energy 25764.207315444946


1it [31:58, 1918.24s/it]

Trainning will be on cuda
MAE tensor(0.1053, device='cuda:0') MSE tensor(0.0169, device='cuda:0') RMSE tensor(0.1301, device='cuda:0') RunningTime 2.6645817214792427 Energy 28234.775030612946


2it [2:04:46, 4065.29s/it]

...

3it [2:46:10, 3343.24s/it]

Trainning will be on cuda
MAE tensor(0.1117, device='cuda:0') MSE tensor(0.0233, device='cuda:0') RMSE tensor(0.1527, device='cuda:0') RunningTime 2.8004946040215892 Energy 34557.29510319233


4it [3:26:40, 2982.75s/it]

Trainning will be on cuda
MAE tensor(0.0678, device='cuda:0') MSE tensor(0.0097, device='cuda:0') RMSE tensor(0.0984, device='cuda:0') RunningTime 2.925456549607071 Energy 32044.75672507286


5it [4:06:07, 2953.43s/it]
